# Strava API Tutorial

## Importing libaries

In [1]:
import pandas as pd
import requests
import json

## Storing API credentials

In [6]:
api_credentials = json.load(open('.secret/strava_api_credentials.json', 'r'))
client_id = api_credentials['client_id']
client_secret = api_credentials['client_secret']
access_token = api_credentials['access_token']
refresh_token = api_credentials['refresh_token']

## Storing request fundementals

In [7]:
base_url = "https://www.strava.com/api/v3"
headers = {"Authorization": "Bearer {}".format(access_token)}

## Client Authentication

In [4]:
# authorisation_code = "ad9c4b97c1bcb1e9ea035e95aa2081691bf53f73"
# req = requests.post("https://www.strava.com/oauth/token?client_id={}&client_secret={}&code={}&grant_type=authororization_code".format(client_id, client_secret, refresh_token))
req = requests.post("https://www.strava.com/oauth/token?client_id={}&client_secret={}&refresh_token={}&grant_type=refresh_token".format(client_id, client_secret, refresh_token))
req.json()

{'token_type': 'Bearer',
 'access_token': '9ea5041b0cd476de23a667d71021ad07f716468b',
 'expires_at': 1582748747,
 'expires_in': 21600,
 'refresh_token': 'f437acf70f96a2e3cadbc2416b63f2855fa9dfee'}

# Getting basic activity information

In [9]:
req_1 = requests.get("{}/athlete/activities".format(base_url), headers = headers)
req_1.json()[0]

{'resource_state': 2,
 'athlete': {'id': 34272639, 'resource_state': 1},
 'name': 'Tuesday intervals (including WU/WD)',
 'distance': 11244.2,
 'moving_time': 3206,
 'elapsed_time': 4038,
 'total_elevation_gain': 56.2,
 'type': 'Run',
 'workout_type': 0,
 'id': 3131802476,
 'external_id': 'garmin_push_4590785536',
 'upload_id': 3345747701,
 'start_date': '2020-02-25T18:34:37Z',
 'start_date_local': '2020-02-25T18:34:37Z',
 'timezone': '(GMT+00:00) Europe/London',
 'utc_offset': 0.0,
 'start_latlng': [51.822253, -0.203645],
 'end_latlng': [51.821375, -0.202674],
 'location_city': None,
 'location_state': None,
 'location_country': None,
 'start_latitude': 51.822253,
 'start_longitude': -0.203645,
 'achievement_count': 10,
 'kudos_count': 14,
 'comment_count': 0,
 'athlete_count': 15,
 'photo_count': 0,
 'map': {'id': 'a3131802476',
  'summary_polyline': 'aph{Hxwf@@qALe@\\a@f@SvAMLMFSB_AFMvBc@lDgCt@w@z@s@~AgB|AyBd@[BG?g@HYTa@XODMH_CbA_J@sAD}@f@gDtAmDfAoAr@a@x@IxALhAh@Zf@`@VxAfB`Al@lBt@rB

### Looping over all user activities

In [10]:
activities = []
i = 1
# first request
req = requests.get("{}/athlete/activities".format(base_url), headers = headers, params = {'page': i})
# looping over user activities in batches of 30 until request is empty
while len(req.json()) > 0:
    # storing attributes from each activity in a dictionary and adding to a list
    for activity in req.json():
        # extracting attributes from request, padding missing values with empty strings
        activity_name, activity_id, activity_type = activity['name'], activity['id'], activity['type']
        distance, time = activity.get('distance', ''), activity.get('elapsed_time', '')
        elevation_gain = activity.get('total_elevation_gain', '')
        kudos = activity.get('kudos_count', '')
        start_date = activity.get('start_date', '')
        average_speed, max_speed, average_cadence = activity.get('average_speed', ''), activity.get('max_speed', ''), activity.get('average_cadence', '')
        average_hr, max_hr = activity.get('average_heartrate', ''), activity.get('max_heartrate', '')
        suffer_score = activity.get('suffer_score', '')
        # storing attributes in a dictionary
        activity_attributes = {'activity_name': activity_name, 'activity_id': activity_id, 'activity_type': activity_type, 'distance': distance, 'time': time, 'elevation_gain': elevation_gain, 'kudos': kudos, 'start_date': start_date, 'average_speed': average_speed, 'max_speed': max_speed,
'average_cadence': average_cadence, 'average_hr': average_hr, 'max_hr': max_hr, 'suffer_score': suffer_score}
        # appending dictionary to list
        activities.append(activity_attributes)
    # iterating to next page
    i += 1
    req = requests.get("{}/athlete/activities".format(base_url), headers = headers, params = {'page': i})

### Storing running activities in a dataframe

In [11]:
activities_df = pd.DataFrame(activities)
runs_df = activities_df.loc[activities_df['activity_type'] == 'Run']

In [13]:
runs_df.to_csv('run_activities.csv')

In [17]:
runs_df = pd.read_csv('run_activities.csv').drop(columns = ['Unnamed: 0', 'activity_type'])
runs_df.head()

,activity_id,activity_name,average_cadence,average_hr,average_speed,distance,elevation_gain,kudos,max_hr,max_speed,start_date,suffer_score,time
0,3131802476,Tuesday intervals (including WU/WD),84.5,153.3,3.507,11244.2,56.2,14,186.0,7.4,2020-02-25T18:34:37Z,69.0,4038
1,3124325868,Morning Run - Welwyn,81.9,140.3,3.514,9944.3,118.4,10,170.0,5.2,2020-02-23T08:34:33Z,34.0,2830
2,3121033650,Panshanger PR (18:11 - 1st),85.7,173.4,4.490,5006.7,36.2,16,195.0,7.1,2020-02-22T09:02:55Z,58.0,1238
3,3121032425,PR WU,81.0,141.0,3.347,2604.2,2.1,1,158.0,5.3,2020-02-22T08:42:49Z,9.0,778
4,3117367456,Track,84.0,151.5,3.842,11187.1,0.0,11,185.0,6.7,2020-02-20T18:35:22Z,65.0,3383


### Storing activity ids

In [18]:
activity_ids = list(runs_df['activity_id'])

# Getting activity split times

In [14]:
req_2 = requests.get("{}/activities/{}/laps".format(base_url, activity_ids[1]), headers = headers)
req_2.json()

[{'id': 10324732285,
  'resource_state': 2,
  'name': 'Lap 1',
  'activity': {'id': 3101510285, 'resource_state': 1},
  'athlete': {'id': 34272639, 'resource_state': 1},
  'elapsed_time': 218,
  'moving_time': 218,
  'start_date': '2020-02-15T09:02:17Z',
  'start_date_local': '2020-02-15T09:02:17Z',
  'distance': 1000.0,
  'start_index': 0,
  'end_index': 219,
  'total_elevation_gain': 9.0,
  'average_speed': 4.59,
  'max_speed': 5.5,
  'average_cadence': 87.1,
  'average_heartrate': 135.6,
  'max_heartrate': 147.0,
  'lap_index': 1,
  'split': 1,
  'pace_zone': 5},
 {'id': 10324732295,
  'resource_state': 2,
  'name': 'Lap 2',
  'activity': {'id': 3101510285, 'resource_state': 1},
  'athlete': {'id': 34272639, 'resource_state': 1},
  'elapsed_time': 225,
  'moving_time': 225,
  'start_date': '2020-02-15T09:05:57Z',
  'start_date_local': '2020-02-15T09:05:57Z',
  'distance': 1000.0,
  'start_index': 220,
  'end_index': 445,
  'total_elevation_gain': 20.0,
  'average_speed': 4.44,
  'ma

In [15]:
activity_splits = []
i = 1
for split in req_2.json():
    if split['distance'] > 250:
        activity_id, split_index = split['activity']['id'], i
        split_time = split['elapsed_time']
        split_distance, split_elevation_gain = split['distance'], split['total_elevation_gain']
        split_average_speed, split_max_speed = split['average_speed'], split['max_speed']
        split_average_hr, split_max_hr = split['average_heartrate'], split['max_heartrate']
        split_average_cadence = split['average_cadence']
        split_attributes = {'activity_id': activity_id, 'split_index': split_index, 'split_time': split_time, 'split_distance': split_distance, 'split_elevation_gain': split_elevation_gain, 'split_average_speed': split_average_speed, 'split_max_speed': split_max_speed, 'split_average_hr': split_average_hr, 'split_max_hr': split_max_hr, 'split_average_cadence': split_average_cadence}
        activity_splits.append(split_attributes)
        i += 1
    else:
        continue
activity_splits_df = pd.DataFrame.from_dict(activity_splits)

# Getting activity HR zones

In [67]:
req_3 = requests.get("{}/activities/{}/zones".format(base_url, str(req_1.json()[0]['id'])), headers = new_headers)

In [68]:
req_3.json()

[{'score': 62,
  'distribution_buckets': [{'max': 123, 'min': 0, 'time': 21},
   {'max': 153, 'min': 123, 'time': 10},
   {'max': 169, 'min': 153, 'time': 234},
   {'max': 184, 'min': 169, 'time': 367},
   {'max': -1, 'min': 184, 'time': 442}],
  'type': 'heartrate',
  'resource_state': 3,
  'sensor_based': True,
  'points': 46,
  'custom_zones': False},
 {'score': 10,
  'distribution_buckets': [{'max': 3.270811744386874, 'min': 0, 'time': 0},
   {'max': 3.799913644214162, 'min': 3.270811744386874, 'time': 0},
   {'max': 4.232815198618307, 'min': 3.799913644214162, 'time': 4},
   {'max': 4.521416234887737, 'min': 4.232815198618307, 'time': 362},
   {'max': 4.810017271157167, 'min': 4.521416234887737, 'time': 558},
   {'max': -1, 'min': 4.810017271157167, 'time': 150}],
  'type': 'pace',
  'resource_state': 3,
  'sensor_based': True}]